# Code to convert XML labels to TXT labels to be compatible with yolo

In [ ]:
import os
import xml.etree.ElementTree as ET

# Rutas
xmls_dir = "data/labels_parte"  # Carpeta donde están los XMLs
yolo_dir = "yolo"      # Carpeta donde se guardarán los archivos YOLO
clases = ["number_plate"]  # Reemplaza con tus clases definidas

# Crear carpeta YOLO si no existe
os.makedirs(yolo_dir, exist_ok=True)

# Función para convertir coordenadas a formato YOLO
def convertir_a_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_centro = (box[0] + box[2]) / 2.0
    y_centro = (box[1] + box[3]) / 2.0
    ancho = box[2] - box[0]
    alto = box[3] - box[1]
    return x_centro * dw, y_centro * dh, ancho * dw, alto * dh

# Procesar XMLs
for archivo in os.listdir(xmls_dir):
    if archivo.endswith(".xml"):
        xml_path = os.path.join(xmls_dir, archivo)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Obtener tamaño de la imagen
        size = root.find("size")
        ancho = int(size.find("width").text)
        alto = int(size.find("height").text)

        # Procesar objetos
        yolo_anotaciones = []
        for obj in root.findall("object"):
            nombre_clase = obj.find("name").text
            if nombre_clase not in clases:
                continue  # Ignorar clases no definidas
            id_clase = clases.index(nombre_clase)

            # Obtener coordenadas del bounding box
            bndbox = obj.find("bndbox")
            xmin = float(bndbox.find("xmin").text)
            ymin = float(bndbox.find("ymin").text)
            xmax = float(bndbox.find("xmax").text)
            ymax = float(bndbox.find("ymax").text)

            # Convertir coordenadas a formato YOLO
            x_centro, y_centro, ancho_norm, alto_norm = convertir_a_yolo((ancho, alto), (xmin, ymin, xmax, ymax))
            yolo_anotaciones.append(f"{id_clase} {x_centro:.6f} {y_centro:.6f} {ancho_norm:.6f} {alto_norm:.6f}")

        # Guardar archivo YOLO
        yolo_path = os.path.join(yolo_dir, os.path.splitext(archivo)[0] + ".txt")
        with open(yolo_path, "w") as yolo_file:
            yolo_file.write("\n".join(yolo_anotaciones))

print(f"Conversión completada. Archivos YOLO guardados en '{yolo_dir}'.")


# Code to validate image/label pairs existence

In [ ]:
import os
import shutil

# Rutas de entrada y salida
imagenes_dir = "data/dataset"  # Cambia esto a la ruta de tu carpeta de imágenes
xmls_dir = "data/labels_parte"         # Cambia esto a la ruta de tu carpeta de XMLs
dataset_dir = "filtered_dataset"           # Cambia esto a la ruta donde quieres las imágenes válidas
labels_dir = "filtered_labels"             # Cambia esto a la ruta donde quieres las anotaciones XML válidas

# Crear carpetas de salida si no existen
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Extensiones válidas
extensiones_imagenes = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}  # Agrega más si es necesario

# Obtener listas de nombres de archivos base (sin extensiones)
imagenes_nombres = {os.path.splitext(f)[0] for f in os.listdir(imagenes_dir) 
                    if os.path.splitext(f)[1].lower() in extensiones_imagenes}
xmls_nombres = {os.path.splitext(f)[0] for f in os.listdir(xmls_dir) if f.endswith(".xml")}

# Encontrar nombres comunes entre imágenes y XMLs
nombres_comunes = imagenes_nombres & xmls_nombres

# Copiar archivos a las nuevas carpetas
for nombre in nombres_comunes:
    imagen_path = os.path.join(imagenes_dir, nombre + ".jpg")  # Cambiar extensión según tus imágenes
    xml_path = os.path.join(xmls_dir, nombre + ".xml")
    
    # Copiar imagen
    for ext in extensiones_imagenes:
        posible_imagen = os.path.join(imagenes_dir, nombre + ext)
        if os.path.exists(posible_imagen):
            shutil.copy2(posible_imagen, os.path.join(dataset_dir, os.path.basename(posible_imagen)))
            break

    # Copiar XML
    shutil.copy2(xml_path, os.path.join(labels_dir, os.path.basename(xml_path)))

print(f"Se copiaron {len(nombres_comunes)} pares de archivos a las carpetas '{dataset_dir}' y '{labels_dir}'.")
